In [1]:
from pyspark.sql import functions as f
from pyspark.sql import types as t
from pyspark.sql.window import Window
import dateutil.parser 

In [2]:
countryCode = 'DE'

In [3]:
sale  =  spark.table("dev_sources_ohub.cleaned_operator_sales").where(f.col('countryCode') == countryCode).where(f.col('transactionDate').isNotNull())

def dateConverter(isoDate):
  dateOutput = dateutil.parser.parse(isoDate).strftime('%y-%m-%d')
  return dateOutput

dateConverterUDF = f.udf(lambda x: dateConverter(x))
sale = sale.where(f.col('transactionDate').rlike("[0-9]"))
sale = sale.withColumn('date', f.date_format(f.to_date(f.col('transactionDate')), 'yyyyMMdd'))
sale = sale.where(f.col('date') >= '20190101')
#split_col = f.split(sale['operatorConcatID'], '~')
#sale = sale.withColumn('clientID',split_col.getItem(2))
# sale = sale.join(dfClientId, on = ['operatorConcatID'], how = 'left_outer').\
# withColumn('clientIDInt', f.col('clientID').cast('bigint')).\
# withColumn('clientID', f.when(f.col('clientIDInt').isNotNull(), f.col('clientIDInt')).otherwise(f.col('clientID')))

operatorSalesValue = sale.where(f.col('amount') > 0).where(f.col('cuEanCode').isNotNull()).groupBy('operatorOhubId').agg(f.sum(f.col('quantityOfUnits')*((f.col('cuListingPriceInCents')/f.lit(100)).cast('double'))).alias('operatorSalesValue'), f.countDistinct(f.col('cuEanCode')).alias('grab'))

In [4]:
display(operatorSalesValue)

operatorOhubId,operatorSalesValue,grab
fc11d754-8a19-3d26-a9b6-daf386c8e63d,31.41,4
1194f861-1006-4a7e-b8e1-b1a86f68d237,516.74,14
13db8fd0-d424-4b6f-83f4-6a2a12d1b63c,1954.36,6
6ff2054c-37ba-405e-af00-3b47a5faf71b,1176096.2699999998,260
10797825-bfa3-4b3e-982e-ab3984bef5e0,1387.0299999999997,9
251f4840-5c4a-44e5-9f2f-da08becca5d3,9351.3,66
f259b98b-6f55-430d-9923-97d4f86705a9,3777.9399999999996,23
d536ca61-993c-49f2-be99-419b207618b4,11558.3,32
8f3cf43a-48c6-4b3c-8484-fb43f3817890,45199.110000000015,46
bf252928-874f-3a9b-9c8e-3c92d2cdc81a,87.84,1


#### adding propensity to buy

In [6]:
relevantProductsValueOperators.count()

Out[6]: 318070

In [7]:
display(relevantProductsValueOperators)

operatorID,osmid,placeIdGoogle,operatorOhubID,name,address,postalCode,city,latitude,longitude,businessType,website,phone,nameGoogle,addressGoogle,postalCodeGoogle,cityGoogle,latitudeGoogle,longitudeGoogle,businessTypeGoogle,websiteGoogle,type,uid,user_sid,nameOSM,addressOSM,postalCodeOSM,cityOSM,latitudeOSM,longitudeOSM,businessTypeOSM,websiteOSM,cuisineTypeOSM,phoneOSM,nameOHUB,addressOHUB,zipcodeOHUB,cityOHUB,operatorConcatID,channelOHUB,source,globalListChannels,globalChannel,cuisineType,chain,d_buyer,cuEanCode,rationale,productName
/ / 0089343a-76b1-30e4-8c32-1f1402535df1,null,null,0089343a-76b1-30e4-8c32-1f1402535df1,COMERCIAL GUSSTET,HOLANDA,08228,Les Fonts (terrassa),null,null,Otros,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,COMERCIAL GUSSTET,HOLANDA,08228,Les Fonts (terrassa),ES~GESCON~22260294,Otros,OHUB,[Other],Other,us,Unknown,1,null,Sales Data,null
/ / 0089343a-76b1-30e4-8c32-1f1402535df1,null,null,0089343a-76b1-30e4-8c32-1f1402535df1,COMERCIAL GUSSTET,HOLANDA,08228,Les Fonts (terrassa),null,null,Otros,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,COMERCIAL GUSSTET,HOLANDA,08228,Les Fonts (terrassa),ES~GESCON~22260294,Otros,OHUB,[Other],Other,us,Unknown,1,8712100410703,Sales Data,knorr caldo líquido concentrado de carne sin gluten botella 1l
/ / 0089343a-76b1-30e4-8c32-1f1402535df1,null,null,0089343a-76b1-30e4-8c32-1f1402535df1,COMERCIAL GUSSTET,HOLANDA,08228,Les Fonts (terrassa),null,null,Otros,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,COMERCIAL GUSSTET,HOLANDA,08228,Les Fonts (terrassa),ES~GESCON~22260294,Otros,OHUB,[Other],Other,us,Unknown,1,8410127127900,Sales Data,"maizena harina fina de maiz espesante sin gluten caja 2,5 kg"
/ / 02a07098-7f85-3fb9-9370-2be1801f97dc,null,null,02a07098-7f85-3fb9-9370-2be1801f97dc,CAFETERIA CARLOS,MANCHA 0002,03183,Torrevieja,null,null,Restaurante menú/carta,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,CAFETERIA CARLOS,MANCHA 0002,03183,Torrevieja,ES~GESCON~30093909,Restaurante menú/carta,OHUB,"[Other, Restaurant, Cafe]",Restaurant,Unknown,Unknown,1,8410127183012,Sales Data,knorr bovril caldo concentrado de carne bote 500g
/ / 02a07098-7f85-3fb9-9370-2be1801f97dc,null,null,02a07098-7f85-3fb9-9370-2be1801f97dc,CAFETERIA CARLOS,MANCHA 0002,03183,Torrevieja,null,null,Restaurante menú/carta,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,CAFETERIA CARLOS,MANCHA 0002,03183,Torrevieja,ES~GESCON~30093909,Restaurante menú/carta,OHUB,"[Other, Restaurant, Cafe]",Restaurant,Unknown,Unknown,1,8712566192038,Sales Data,knorr salsa bechamel deshidratada bote 715g
/ / 02a07098-7f85-3fb9-9370-2be1801f97dc,null,null,02a07098-7f85-3fb9-9370-2be1801f97dc,CAFETERIA CARLOS,MANCHA 0002,03183,Torrevieja,null,null,Restaurante menú/carta,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,CAFETERIA CARLOS,MANCHA 0002,03183,Torrevieja,ES~GESCON~30093909,Restaurante menú/carta,OHUB,"[Other, Restaurant, Cafe]",Restaurant,Unknown,Unknown,1,8412200329508,Sales Data,knorr caldo de pollo deshidratado bote 1kg
/ / 02a07098-7f85-3fb9-9370-2be1801f97dc,null,null,02a07098-7f85-3fb9-9370-2be1801f97dc,CAFETERIA CARLOS,MANCHA 0002,03183,Torrevieja,null,null,Restaurante menú/carta,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,CAFETERIA CARLOS,MANCHA 0002,03183,Torrevieja,ES~GESCON~30093909,Restaurante menú/carta,OHUB,"[Other, Restaurant, Cafe]",Restaurant,Unknown,Unknown,1,null,Sales Data,null
/ / 02a07098-7f85-3fb9-9370-2be1801f97dc,null,null,02a07098-7f85-3fb9-9370-2be1801f97dc,CAFETERIA CARLOS,MANCHA 0002,03183,Torrevieja,null,null,Restaurant

In [8]:
relevantProductsValueOperators = spark.table('dev_derived_ouniverse.output_total_universe_' + countryCode)
relevantProductsValueOperators = relevantProductsValueOperators.withColumnRenamed('ohubId','operatorOhubId')

propensityScore    = spark.table('data_user_hien.propensity_model_de_dum_density_urbanisation_competition_discrete').withColumnRenamed('score','propensityScore').drop('label')

operatorValuePropensity      = relevantProductsValueOperators.join(propensityScore,on= ['operatorId'],how = 'left_outer')

operatorValue = operatorValuePropensity.\
join(operatorSalesValue, on = ['operatorOhubId'], how = 'left_outer').distinct()

# operatorValuePropensity      = operatorValuePropensity.withColumn('finalScore',f.col("grab_measurement_score")*f.col("propensityScore"))


validatePropensityScore = operatorValue.\
select('operatorid', 'propensityScore', 'operatorSalesValue', f.when(f.col('operatorSalesValue') != 0 ,1).otherwise(0).alias('buyer')).distinct()

w1 = Window.orderBy(f.col('propensityScore').asc())
                                                               
validatePropensityScore = validatePropensityScore.where(f.col('propensityScore').isNotNull()).withColumn("decilePropensityScore", f.ntile(5).over(w1))

display(validatePropensityScore.groupBy(f.col('decilePropensityScore')).\
agg(
    f.countDistinct(f.col('operatorID')), \
    f.min(f.col('propensityScore')), \
    f.max(f.col('propensityScore')), \
    f.avg(f.col('operatorSalesValue')),\
    f.avg(f.col('buyer'))))

decilePropensityScore,count(operatorID),min(propensityScore),max(propensityScore),avg(operatorSalesValue),avg(buyer)
1,85972,0.2862440158568031,0.4052036824825668,1923.2877605673289,0.12445912622714372
2,85971,0.4052036824825668,0.45221734605085195,1529.1409837170183,0.20688371660211002
3,85971,0.45221734605085195,0.48161117039195733,1328.3128426961332,0.2601807586279094
4,85971,0.48161117039195733,0.5056137278035436,1202.3092796204664,0.27938490886461714
5,85971,0.5056137278035436,0.5600650760678981,909.0970050478813,0.3133033232136418


#### Validation of propensity score, product to menu fit & value tier

##### propensity score

In [11]:
print(operatorValue.columns)

['clientID', 'operatorID', 'osmID', 'PlaceIDGoogle', 'name', 'address', 'postalCode', 'city', 'latitude', 'longitude', 'businessType', 'Website', 'NameGoogle', 'addressGoogle', 'PostalCodeGoogle', 'CityGoogle', 'businessTypeGoogle', 'WebsiteGoogle', 'LatitudeGoogle', 'LongitudeGoogle', 'type', 'uid', 'nameOSM', 'AddressOSM', 'PostalCodeOSM', 'businessTypesOSM', 'cityOSM', 'cuisineTypeOSM', 'websiteOSM', 'LatitudeOSM', 'LongitudeOSM', 'clientName', 'clientNameAlternative', 'clientCity', 'clientAddress', 'clientPostalcode', 'clientNIP', 'source', 'cuisineType', 'globalListChannels', 'globalChannel', 'd_buyer', 'clientIDInt', 'totalProductPrice', 'avgTotalProductPriceNoProducts', 'avgGrab', 'propensityScore', 'finalScore', 'operatorSalesValue', 'grab', 'finalScoreCorrected', 'valueTier']

##### product to menu fit

In [13]:
validateProductToMenuFit = universe.\
join(operatorProductToMenuFit, on = ['operatorID'], how = 'left_outer').\
join(operatorSalesValue, on = ['clientID'], how = 'left_outer').\
join(propensityScore, on = ['operatorID'], how = 'left_outer').\
select('operatorid', 'productToMenuFit', 'operatorSalesValue','grab','propensityScore',f.when(f.col('operatorSalesValue').isNotNull(),1).otherwise(0).alias('buyer')).distinct()

validateProductToMenuFit = validateProductToMenuFit.withColumn('relevantProducts',f.when(f.col('grab').isNotNull(),f.col('grab')).otherwise(f.col('productToMenuFit')))
validateProductToMenuFit = validateProductToMenuFit.crossJoin(relevantProductsGrabOperators).\
withColumn('relevantProducts',f.when(f.col('relevantProducts').isNotNull(),f.col('relevantProducts')).otherwise(f.col('avgGrab')))

validateProductToMenuFit = validateProductToMenuFit.withColumn('valueTier',f.col('relevantProducts')*f.col('propensityScore'))

w = Window.orderBy(f.col('valueTier'))
                                                               
validateProductToMenuFit = validateProductToMenuFit.where(f.col('valueTier').isNotNull()).withColumn("decileProductToMenuFit", f.ntile(5).over(w))

validateProductToMenuFit.groupBy(f.col('decileProductToMenuFit')).\
agg(f.countDistinct(f.col('operatorID')), \
    f.min(f.col('valueTier')), \
    f.max(f.col('valueTier')), \
    f.avg(f.col('operatorSalesValue')),\
    f.avg(f.col('buyer'))).\
show(truncate=False, n=20)


--------------------------------------------------------------------------- 
 NameError Traceback (most recent call last)
 <command-1249654934722653> in <module> 
 ----> 1 validateProductToMenuFit = universe . \ 
 2 join ( operatorProductToMenuFit , on = [ 'operatorID' ] , how = 'left_outer' ) . \ 
 3 join ( operatorSalesValue , on = [ 'clientID' ] , how = 'left_outer' ) . \ 
 4 join ( propensityScore , on = [ 'operatorID' ] , how = 'left_outer' ) . \ 
 5 select ( 'operatorid' , 'productToMenuFit' , 'operatorSalesValue' , 'grab' , 'propensityScore' , f . when ( f . col ( 'operatorSalesValue' ) . isNotNull ( ) , 1 ) . otherwise ( 0 ) . alias ( 'buyer' ) ) . distinct ( ) 

 NameError : name 'universe' is not defined

In [14]:
print(validateProductToMenuFit.count(), validateProductToMenuFit.select('operatorID').distinct().count())

##### value Tier

In [16]:
validateValueTier = operatorValue.\
select('operatorid', 'totalProductPrice', 'operatorSalesValue', f.when(f.col('operatorSalesValue') != 0, 1).otherwise(0).alias('buyer')).distinct()

w = Window.orderBy(f.col('totalProductPrice'))
                                                               
validateValueTier = validateValueTier.where(f.col('totalProductPrice').isNotNull()).withColumn("valueTier", f.ntile(10).over(w))

validateValueTier.groupBy(f.col('valueTier')).\
agg(f.countDistinct(f.col('operatorID')), \
    f.min(f.col('totalProductPrice')), \
    f.max(f.col('totalProductPrice')), \
    f.avg(f.col('operatorSalesValue')),\
    f.avg(f.col('buyer'))).\
orderBy(f.col('valueTier').asc()).\
show(truncate=False, n=20)


In [17]:
validateValueTier = operatorValue.\
select('operatorid', 'finalScore', 'operatorSalesValue', f.when(f.col('operatorSalesValue') != 0, 1).otherwise(0).alias('buyer')).distinct()

w = Window.orderBy(f.col('finalScore'))
                                                               
validateValueTier = validateValueTier.where(f.col('finalScore') != 0).where(f.col('finalScore').isNotNull()).withColumn("valueTier", f.ntile(5).over(w))

validateValueTier.groupBy(f.col('valueTier')).\
agg(f.countDistinct(f.col('operatorID')), \
    f.min(f.col('finalScore')), \
    f.max(f.col('finalScore')), \
    f.avg(f.col('operatorSalesValue')),\
    f.avg(f.col('buyer'))).\
orderBy(f.col('valueTier').asc()).\
show(truncate=False, n=20)


In [18]:
print(operatorValue.columns)

In [19]:
validateValueTier = operatorValue.\
select('operatorid', 'finalScoreCorrected', 'operatorSalesValue', f.when(f.col('clientID').isNotNull(),1).otherwise(0).alias('buyer')).distinct()

w = Window.orderBy(f.col('finalScoreCorrected'))
                                                               
validateValueTier = validateValueTier.where(f.col('finalScoreCorrected').isNotNull()).withColumn("valueTier", f.ntile(5).over(w))

validateValueTier.groupBy(f.col('valueTier')).\
agg(f.countDistinct(f.col('operatorID')), \
    f.min(f.col('finalScoreCorrected')), \
    f.max(f.col('finalScoreCorrected')), \
    f.avg(f.col('operatorSalesValue')),\
    f.avg(f.col('buyer'))).\
orderBy(f.col('valueTier').asc()).\
show(truncate=False, n=20)


In [20]:
validateValueTier = operatorValue.where(f.col('rationale') != 'Sales Data').\
select('operatorid', 'finalScoreCorrected', 'operatorSalesValue', f.when(f.col('operatorSalesValue') != 0 ,1).otherwise(0).alias('buyer')).distinct()

w = Window.orderBy(f.col('finalScore'))
                                                               
validateValueTier = validateValueTier.where(f.col('finalScore').isNotNull()).withColumn("valueTier", f.ntile(5).over(w))

validateValueTier.groupBy(f.col('valueTier')).\
agg(f.countDistinct(f.col('operatorID')), \
    f.min(f.col('finalScore')), \
    f.max(f.col('finalScore')), \
    f.avg(f.col('operatorSalesValue')),\
    f.avg(f.col('buyer'))).\
orderBy(f.col('valueTier').asc()).\
show(truncate=False, n=20)


In [21]:
print(validateValueTier.count(), validateValueTier.select('operatorID').distinct().count())

### Save value tier in delta table

In [23]:

# hiveTable = 'dev_derived_eotm.output_valuetier_pl'
# deltaTable = '/mnt/datamodel/dev/derived/ovalue/output/valueTier_pl' 

# operatorValue.write \
#   .format("delta") \
#   .mode("overwrite") \
#   .option("overwriteSchema", "true") \
#   .save(deltaTable)

# sqlQuery1 = "drop table if exists " + hiveTable
# sqlQuery2 = "create table " + hiveTable + " using delta location '" + deltaTable + "'"
# spark.sql(sqlQuery1)
# spark.sql(sqlQuery2)
